In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
from synth_helpers import * 
import datetime
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())
from statsmodels.distributions.empirical_distribution import ECDF
import copy
import pickle

## Import data 

In [ ]:
# Linear regression with ridge
# method='linreg'
# daily = 'weekly'
# ridge = True

# Linear regression without ridge
# method='linreg'
# daily = 'weekly'
# ridge = False

# RSC
# daily = 'weekly'
# method='RSC'
# ridge = False
ridge_name ='ridge' if ridge else 'noridge'

In [47]:
exp_name = daily+'-'+method+'-morecities-'+ridge_name#-neg-allcats'
print exp_name
res = pickle.load(open('results/'+exp_name+'-sc-out.p', 'rb'))
locals().update(res)
# print_cats = [ x.replace('_', ' ') for x in cats ]


weekly-RSC-morecities-noridge


In [48]:
n_placebos_

array([25., 25., 25., 25., 19.])

In [25]:
Rsquareds


array([0.73762508, 0.68842555, 0.72879319, 0.5618315 , 0.82424032])

In [49]:
def latex_float(float_str):
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str
    
cols = ['ATE (/1000)', 'p (ATE)', 'p (RMSE)', r'Pre-tx $R^2$', 'Pre-tx RMSE','Avg. plac. RMSE', 'Num. plac.' ]
# print_cats = [x.replace('_',' ') for x in cats]
print_cats = cats_print
print ''+"&" + " & ".join([  cols[j] for j in range(len(cols))])  + '\\\\ \hline'
for j in range(len(cats)): 
    includes=[str(np.round(placebo_ate[j]*1000,4)), # rescale by city population
#         latex_float(np.format_float_scientific(placebo_ate[j],2) ),
              str(np.round(1-placebo_ate_p[j],2)), 
              str(np.round(1-placebo_mse_ratio_p[j],2)),
              str(np.round(Rsquareds[j],2)),
              latex_float(np.format_float_scientific(SC_mses[j],2)),
              latex_float(np.format_float_scientific(np.mean(placebo_mses[j]),2)), 
             str(n_placebos_.astype(int)[j])]    
    print print_cats[j]+ "&" + " & ".join(['$'+x+'$' for x in includes])  + '\\\\'



# print ''+"&" + " & ".join([ '$'+ cats[j]+'$' for j in range(len(cats))])  + '\\\\'
# print 'Pseudo-p-value (ATE)'+ "&" + " & ".join([ '$'+ str(np.round(placebo_ate_p[j],2))+'$' for j in range(len(cats))])  + '\\\\'
# print 'Pre-treatment RMSE'+ "&" + " & ".join([ '$'+ str(np.format_float_scientific(SC_mses[j],2))+'$'for j in range(len(cats))] )  + '\\\\'
# print 'Avg. Placebo RMSE'+ "&" + " & ".join([ '$'+ str(np.format_float_scientific(np.mean(x),2))+'$' for x in placebo_mses_])  + '\\\\'




&ATE (/1000) & p (ATE) & p (RMSE) & Pre-tx $R^2$ & Pre-tx RMSE & Avg. plac. RMSE & Num. plac.\\ \hline
Violent Assault&$0.0163$ & $0.36$ & $0.21$ & $0.76$ & $8.23 \times 10^{-6}$ & $2.07 \times 10^{-5}$ & $25$\\
Property Theft&$0.0197$ & $0.44$ & $0.79$ & $0.72$ & $1.78 \times 10^{-5}$ & $2.71 \times 10^{-5}$ & $25$\\
Property Burglary&$0.0319$ & $0.36$ & $0.38$ & $0.91$ & $3.49 \times 10^{-6}$ & $1.15 \times 10^{-5}$ & $25$\\
Violent Robbery&$0.0287$ & $0.2$ & $0.21$ & $0.72$ & $8.78 \times 10^{-6}$ & $1.65 \times 10^{-5}$ & $25$\\
Drug&$-0.0006$ & $0.68$ & $0.89$ & $0.72$ & $8.36 \times 10^{-6}$ & $2.01 \times 10^{-5}$ & $19$\\


## Multiple testing

In [ ]:
placebo_ate_p

In [34]:
import statsmodels
from statsmodels.stats.multitest import multipletests
one_sided_p =  1 - placebo_ate_p 
sort_order = np.argsort(one_sided_p)
sorted_p = one_sided_p[sort_order]
print sorted_p
res = multipletests(sorted_p, alpha=0.1,method='holm-sidak')
print res[0]
adjusted_p = res[1]
print adjusted_p
print  "&" + " & ".join([ print_cats[sort_order[ind]] for ind in range(len(print_cats))] )  + '\\\\ \hline'
print "Adjusted p (ATE)" + "&" + " & ".join([ '$'+ str(np.round(adjusted_p[ind],2))+'$' for ind in range(len(cats))] )  + '\\\\ '
print "Conclusion" + "&" + " & ".join([ 'Fail to Reject' if not res[0][ind] else 'Reject' for ind in range(len(cats))] ) + '\\\\ '
# mse test statistic
one_sided_p =  1 - placebo_mse_ratio_p 
sort_order = np.argsort(one_sided_p)
sorted_p = one_sided_p[sort_order]

res = multipletests(sorted_p, alpha=0.1,method='holm-sidak')

adjusted_p = res[1]
print "Adjusted p (RMSE)" + "&" + " & ".join([ '$'+ str(np.round(adjusted_p[ind],2))+'$' for ind in range(len(cats))] )  + '\\\\ '
print "Conclusion" + "&" + " & ".join([ 'Fail to Reject' if not res[0][ind] else 'Reject' for ind in range(len(cats))] )
print sorted_p
print res[0]
print adjusted_p



[0.22727273 0.29166667 0.35714286 0.40909091 0.78571429]
[False False False False False]
[0.72449391 0.74826087 0.74826087 0.74826087 0.78571429]
&Violent Robbery & Property Theft & Property Burglary & Violent Assault & Drug\\ \hline
Adjusted p (ATE)&$0.72$ & $0.75$ & $0.75$ & $0.75$ & $0.79$\\ 
Conclusion&Fail to Reject & Fail to Reject & Fail to Reject & Fail to Reject & Fail to Reject\\ 
Adjusted p (RMSE)&$0.5$ & $0.5$ & $0.54$ & $0.56$ & $0.56$\\ 
Conclusion&Fail to Reject & Fail to Reject & Fail to Reject & Fail to Reject & Fail to Reject
[0.13043478 0.15384615 0.23076923 0.33333333 0.52380952]
[False False False False False]
[0.50282326 0.50282326 0.54483386 0.55555556 0.55555556]


## Table for in-time placebos

In [50]:
cols = ['Category', 'Start date',r'ATE (/1000)', ' p (ATE)', 'p (RMSE)', 'Avg. Placebo RMSE']
# print_cats = [x.replace('_',' ') for x in cats]
print  " & ".join([  cols[j] for j in range(len(cols))])  + '\\\\ \hline'


for j in range(len(cats)): 
    for k in range(len(alt_end_times)):
    
        includes=[str(np.round(placebo_intime_ate_nyc[j][k]*1000,4)),
#             latex_float(np.format_float_scientific(placebo_intime_ate_nyc[j][k],2)),
                  str(np.round(1-placebo_intime_ate_p[j][k],2)), 
                  str(np.round(1-placebo_intime_mse_ratio_p[j][k],2)),
                  latex_float(np.format_float_scientific(np.mean(placebo_intime_mses_[j][k]),2)) ]    
        cat_ = print_cats[j] if k ==0 else ''
        print cat_ + "&" + datetime.datetime.strptime(alt_end_times[k], '%Y-%m-%d').strftime('%m/%d/%y')+ "&"+ " & ".join(['$'+x+'$' for x in includes])  + '\\\\'




Category & Start date & ATE (/1000) &  p (ATE) & p (RMSE) & Avg. Placebo RMSE\\ \hline
Violent Assault&01/01/19&$-0.0026$ & $0.64$ & $0.92$ & $1.98 \times 10^{-5}$\\
&03/01/19&$0.0049$ & $0.56$ & $0.71$ & $1.99 \times 10^{-5}$\\
&06/01/19&$0.0087$ & $0.48$ & $0.42$ & $2.00 \times 10^{-5}$\\
Property Theft&01/01/19&$-0.0436$ & $0.76$ & $0.62$ & $2.30 \times 10^{-5}$\\
&03/01/19&$-0.0337$ & $0.76$ & $0.62$ & $2.46 \times 10^{-5}$\\
&06/01/19&$0.0016$ & $0.6$ & $1.0$ & $2.50 \times 10^{-5}$\\
Property Burglary&01/01/19&$-0.0016$ & $0.36$ & $0.62$ & $6.38 \times 10^{-6}$\\
&03/01/19&$-0.0005$ & $0.44$ & $0.46$ & $6.50 \times 10^{-6}$\\
&06/01/19&$0.0025$ & $0.4$ & $0.46$ & $6.69 \times 10^{-6}$\\
Violent Robbery&01/01/19&$0.0027$ & $0.2$ & $0.29$ & $6.67 \times 10^{-6}$\\
&03/01/19&$0.0037$ & $0.2$ & $0.29$ & $6.84 \times 10^{-6}$\\
&06/01/19&$0.0052$ & $0.16$ & $0.25$ & $6.8 \times 10^{-6}$\\
Drug&01/01/19&$0.0082$ & $0.37$ & $0.39$ & $1.35 \times 10^{-5}$\\
&03/01/19&$0.0032$ & $0.42$ & 

## Table for early roll-in in-time placebos

In [37]:
placebo_rollin_mse_ratio_p

array([[0.        , 0.        , 0.        ],
       [0.04347826, 0.        , 0.04347826],
       [0.        , 0.13636364, 0.42857143],
       [0.79166667, 0.5       , 0.33333333],
       [0.        , 0.1875    , 0.3125    ]])

In [51]:
cols = ['Category', 'Early Roll-in Start date',r'ATE (/ 1000)', 'p (ATE)','p (RMSE)', 'Avg. Placebo RMSE']
# print_cats = [x.replace('_',' ') for x in cats]
print  " & ".join([  cols[j] for j in range(len(cols))])  + '\\\\ \hline'


for j in range(len(cats)): 
    for k in range(len(alt_end_times)):
    
        includes=[str(np.round(placebo_rollin_ate_nyc[j][k]*1000,4)),
#             latex_float(np.format_float_scientific(placebo_intime_ate_nyc[j][k],2)),
                  str(np.round(1-placebo_rollin_ate_p[j][k],2)), 
                  str(np.round(1-placebo_rollin_mse_ratio_p[j][k],2)), 
                  latex_float(np.format_float_scientific(np.mean(placebo_rollin_mses_[j][k]),2)) ]    
        cat_ = print_cats[j] if k ==0 else ''
        print cat_ + "&" + datetime.datetime.strptime(alt_end_times_rollin[k], '%Y-%m-%d').strftime('%m/%d/%y')+ "&"+ " & ".join(['$'+x+'$' for x in includes])  + '\\\\'





Category & Early Roll-in Start date & ATE (/ 1000) & p (ATE) & p (RMSE) & Avg. Placebo RMSE\\ \hline
Violent Assault&09/01/19&$0.0111$ & $0.44$ & $0.25$ & $2.02 \times 10^{-5}$\\
&10/01/19&$0.0114$ & $0.36$ & $0.25$ & $2.03 \times 10^{-5}$\\
&11/01/19&$0.0139$ & $0.32$ & $0.25$ & $2.05 \times 10^{-5}$\\
Property Theft&09/01/19&$0.0168$ & $0.4$ & $0.96$ & $2.54 \times 10^{-5}$\\
&10/01/19&$0.0141$ & $0.48$ & $0.96$ & $2.56 \times 10^{-5}$\\
&11/01/19&$0.0161$ & $0.52$ & $0.92$ & $2.63 \times 10^{-5}$\\
Property Burglary&09/01/19&$0.0045$ & $0.4$ & $0.42$ & $6.9 \times 10^{-6}$\\
&10/01/19&$0.0057$ & $0.36$ & $0.42$ & $6.89 \times 10^{-6}$\\
&11/01/19&$0.0062$ & $0.32$ & $0.42$ & $6.95 \times 10^{-6}$\\
Violent Robbery&09/01/19&$0.0058$ & $0.2$ & $0.25$ & $6.89 \times 10^{-6}$\\
&10/01/19&$0.0052$ & $0.24$ & $0.25$ & $6.90 \times 10^{-6}$\\
&11/01/19&$0.0051$ & $0.2$ & $0.33$ & $6.92 \times 10^{-6}$\\
Drug&09/01/19&$-0.0004$ & $0.26$ & $0.72$ & $1.29 \times 10^{-5}$\\
&10/01/19&$-0.0004$

## Table with weight information 

In [12]:
print 'City'+"&" + " & ".join([ print_cats[j] for j in range(len(cats))])  + '\\\\ \hline'
mappers=[None]*len(cats)
for j in range(len(cats)): 
    mappers[j] = dict(zip(sc_ctrls[j],range(len(sc_ctrls[j]))))

def check_city_weight(ws, j, i, city, sc_ctrl, mappers): 
    if city in mappers[j].keys(): 
        return str(np.round(ws[j][mappers[j][city]],2))
    else: 
        return '--'
        
for i in range(len(sc_ctrls[0])): #iterate over rows 
    print sc_ctrls[0][i]+ "&" + " & ".join([ '$'+  check_city_weight(ws, j, i, sc_ctrls[0][i], sc_ctrls[j], mappers)   +'$' for j in range(len(cats))])  + '\\\\'


City&Violent Assault & Property Theft & Property Burglary & Violent Robbery & Drug\\ \hline
Austin&$0.04$ & $0.04$ & $0.09$ & $0.06$ & $0.08$\\
Baltimore&$0.01$ & $0.05$ & $0.0$ & $0.01$ & $--$\\
Boston&$0.06$ & $0.05$ & $0.09$ & $0.05$ & $-0.0$\\
Buffalo&$0.02$ & $0.01$ & $-0.08$ & $0.02$ & $--$\\
Chicago&$0.01$ & $0.05$ & $0.08$ & $0.03$ & $0.04$\\
Cincinnati&$0.01$ & $0.07$ & $-0.02$ & $0.01$ & $--$\\
Dallas&$0.06$ & $0.08$ & $0.02$ & $0.05$ & $0.0$\\
Denver&$0.06$ & $-0.01$ & $0.09$ & $0.05$ & $-0.05$\\
Detroit&$-0.01$ & $0.04$ & $-0.01$ & $0.04$ & $-0.1$\\
Houston&$0.02$ & $0.06$ & $0.02$ & $0.05$ & $-0.11$\\
Kansas City&$0.01$ & $0.01$ & $0.01$ & $0.01$ & $-0.0$\\
Little Rock&$0.03$ & $-0.01$ & $-0.04$ & $0.05$ & $--$\\
Los Angeles&$0.06$ & $0.11$ & $0.16$ & $0.06$ & $0.83$\\
Louisville&$0.04$ & $-0.01$ & $0.04$ & $0.05$ & $0.01$\\
Milwaukee&$0.04$ & $0.02$ & $0.02$ & $0.02$ & $--$\\
Nashville&$0.04$ & $0.07$ & $0.03$ & $0.02$ & $0.07$\\
Philadelphia&$0.05$ & $0.05$ & $0.05$ & $0

In [ ]:
sc_ctrls

In [ ]:
method='linreg'
weekly = '5W'
exp_name=weekly+'-'+method

res=pickle.load(open('results/'+exp_name+'-sc-out.p', 'rb'))
locals().update(res)

In [ ]:

# print_cats = [x.replace('_',' ') for x in cats]
# print ''+"&" + " & ".join([  print_cats[j] for j in range(len(cats))])  + '\\\\'
# print 'Pseudo-p-value (ATE)'+ "&" + " & ".join([ '$'+ str(np.round(placebo_ate_p[j],2))+'$' for j in range(len(print_cats))])  + '\\\\'
# print 'Pre-treatment RMSE'+ "&" + " & ".join([ '$'+ latex_float(np.format_float_scientific(SC_mses[j],2))+'$'for j in range(len(cats))] )  + '\\\\'
# print 'Avg. Placebo RMSE'+ "&" + " & ".join([ '$'+ latex_float(np.format_float_scientific(np.mean(x),2))+'$' for x in placebo_mses])  + '\\\\'

cols = ['ATE', 'Pseudo-p (ATE)',r'Pre-tx $R^2$', 'Pre-tx RMSE','Avg. Placebo RMSE' ]
print_cats = [x.replace('_',' ') for x in cats]
print ''+"&" + " & ".join([  cols[j] for j in range(len(cols))])  + '\\\\ \hline'
for j in range(len(cats)): 
    includes=[latex_float(np.format_float_scientific(placebo_ate[j],2)),
              str(np.round(placebo_ate_p[j],2)), 
              str(np.round(Rsquareds[j],2)),
              latex_float(np.format_float_scientific(SC_mses[j],2)),
              latex_float(np.format_float_scientific(np.mean(placebo_mses[j]),2)) ]    
    print print_cats[j]+ "&" + " & ".join(['$'+x+'$' for x in includes])  + '\\\\'


